In [1]:
!pip install -q boto3
!pip install -q xgboost==0.90

tensorflow 2.0.0 requires opt-einsum>=2.3.2, which is not installed.
tensorflow 2.0.0 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
awscli 1.18.11 has requirement botocore==1.15.11, but you'll have botocore 1.15.15 which is incompatible.
awscli 1.18.11 has requirement PyYAML<5.3,>=3.10, but you'll have pyyaml 5.3 which is incompatible.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
tensorflow 2.0.0 requires opt-einsum>=2.3.2, which is not installed.
tensorflow 2.0.0 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
awscli 1.18.11 has requirement botocore==1.15.11, but you'll have botocore 1.15.15 which is incompatible.
awscli 1.18.11 has requirement PyYAML<5.3,>=3.10, but you'll have pyyaml 5.3 which is incompatible.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pi

In [14]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [15]:
prefix_train = 'feature-store/amazon-reviews/csv/scrubbed-tfidf-without-header/train'
prefix_validation = 'feature-store/amazon-reviews/csv/scrubbed-tfidf-without-header/validation'
prefix_test = 'feature-store/amazon-reviews/csv/scrubbed-tfidf-without-header/test'

scrubbed_tfidf_without_header_train_s3_uri = 's3://{}/{}'.format(bucket, prefix_train)
scrubbed_tfidf_without_header_validation_s3_uri = 's3://{}/{}'.format(bucket, prefix_validation)
scrubbed_tfidf_without_header_test_s3_uri = 's3://{}/{}'.format(bucket, prefix_test)

s3_input_train_data = sagemaker.s3_input(s3_data=scrubbed_tfidf_without_header_train_s3_uri, content_type='text/csv')
s3_input_validation_data = sagemaker.s3_input(s3_data=scrubbed_tfidf_without_header_validation_s3_uri, content_type='text/csv')
s3_input_test_data = sagemaker.s3_input(s3_data=scrubbed_tfidf_without_header_test_s3_uri, content_type='text/csv')

print(s3_input_train_data.config)
print(s3_input_validation_data.config)
print(s3_input_test_data.config)

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-835319576252/feature-store/amazon-reviews/csv/scrubbed-tfidf-without-header/train', 'S3DataDistributionType': 'FullyReplicated'}}, 'ContentType': 'text/csv'}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-835319576252/feature-store/amazon-reviews/csv/scrubbed-tfidf-without-header/validation', 'S3DataDistributionType': 'FullyReplicated'}}, 'ContentType': 'text/csv'}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-835319576252/feature-store/amazon-reviews/csv/scrubbed-tfidf-without-header/test', 'S3DataDistributionType': 'FullyReplicated'}}, 'ContentType': 'text/csv'}


In [16]:
!cat src/xgboost_reviews.py

import os
import argparse
import pickle as pkl
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin
import nltk
import re
import xgboost as xgb
from xgboost import XGBClassifier
import glob

# Note:  header=None
def load_dataset(path, sep, header):
    data = pd.concat([pd.read_csv(f, sep=sep, header=header) for f in glob.glob('{}/*.csv'.format(path))], ignore_index = True)

    labels = data.iloc[:,0]
    features = data.drop(data.columns[0], axis=1)
    
    if header==None:
        # Adjust the column names after dropped the 0th column above
        # New column names are 0 (inclusive) to len(features.columns) (exclusive)
        new_column_names = list(range(0, len(features.columns)))
        features.columns = new_column_names

    return features, labels


if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--objective

In [10]:
from sagemaker.xgboost import XGBoost

model_output_path = 's3://{}/models/amazon-reviews/script-mode/training-runs'.format(bucket)

xgb_estimator = XGBoost(entry_point='xgboost_reviews.py', 
                        source_dir='src/',
                        role=role,
                        train_instance_count=2, 
#                        train_instance_type='local',
                        train_instance_type='ml.m4.xlarge',
                        framework_version='0.90-2',
                        py_version='py3',
                        output_path=model_output_path,
                        hyperparameters={'objective':'binary:logistic',
                                         'num_round': 1,
                                         'max_depth': 5},
                        enable_cloudwatch_metrics=True
                       )

### Train the model

In [11]:
xgb_estimator.fit(inputs={'train': s3_input_train_data, 
                          'validation': s3_input_validation_data}, wait=False) 

In [12]:
training_job_name = xgb_estimator.latest_training_job.name
print('training_job_name:  {}'.format(training_job_name))

training_job_name:  sagemaker-xgboost-2020-03-09-05-00-33-097


In [13]:
from sagemaker.xgboost import XGBoost

xgb_estimator = XGBoost.attach(training_job_name=training_job_name)

2020-03-09 05:00:35 Starting - Launching requested ML instances.........
2020-03-09 05:01:42 Starting - Preparing the instances for training......
2020-03-09 05:02:42 Downloading - Downloading input data...
2020-03-09 05:03:24 Training - Downloading the training image..INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Invoking user training script.
INFO:sagemaker-containers:Module xgboost_reviews does not provide a setup.py. 
Generating setup.py
INFO:sagemaker-containers:Generating setup.cfg
INFO:sagemaker-containers:Generating MANIFEST.in
INFO:sagemaker-containers:Installing module with the following command:
/miniconda3/bin/python -m pip install . -r requirements.txt
Processing /opt/ml/code
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449909 sha256=d1b57361e20a447126772aba87c2c2c42c60543b810721981cfd91c44a4d0ba6
  

Wrote model to ./models/script-mode/xgboost-model
Validation AUC:  0.7868888888888889
Validation Accuracy:  0.7868888888888889
Validation Precision:  [0.3115942  0.80192572]
              precision    recall  f1-score   support

           0       0.31      0.05      0.08       907
           1       0.80      0.97      0.88      3593

   micro avg       0.79      0.79      0.79      4500
   macro avg       0.56      0.51      0.48      4500
weighted avg       0.70      0.79      0.72      4500


2020-03-09 05:08:54 Uploading - Uploading generated training model
2020-03-09 05:08:54 Completed - Training job completed
Training seconds: 372
Billable seconds: 372


# Load the Model

In [23]:
!aws s3 ls --recursive $model_output_path

2020-03-09 05:08:53          0 models/amazon-reviews/script-mode/training-runs/sagemaker-xgboost-2020-03-09-05-00-33-097/debug-output/training_job_end.ts


In [17]:
# download the model artifact from AWS S3
!aws s3 cp $model_output_path/$training_job_name/output/model.tar.gz ./models/built-in/

fatal error: An error occurred (404) when calling the HeadObject operation: Key "models/amazon-reviews/script-mode/training-runs/sagemaker-xgboost-2020-03-09-05-00-33-097/output/model.tar.gz" does not exist


# TODO:  Fix prediction

In [ ]:
xgb_endpoint_name = prefix + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

xgb_predictor = xgb_estimator.deploy(
                     initial_instance_count=1, 
                     instance_type='ml.m4.xlarge',
                     endpoint_name=xgb_endpoint_name)

In [ ]:
import boto3
# TODO:  Fix this...
sm_rt = boto3.client('sagemaker-runtime')

# Predict a sample from the validation set
payload = df_validation[:1].drop(['is_positive_sentiment'], axis=1) 
payload = payload.to_csv(header=False, index=False).rstrip()

print(payload)

In [ ]:
response = sm_rt.invoke_endpoint(
    EndpointName=xgb_endpoint_name,
    Body=payload.encode('utf8'),
    ContentType='text/csv')

print(response['Body'].read())

In [ ]:
# Don't forget to delete the endpoint!
# sagemaker_session.delete_endpoint(endpoint_name=xgb_endpoint_name) 

In [ ]:
predictions, raw_outputs = xgb_predictor.predict(["""Very funny. A typical mid 50's comedy."""])
print('Predictions: {}'.format(predictions))
print('Raw outputs: {}'.format(raw_outputs))

In [ ]:
predictions, raw_outputs = xgb_predictor.predict(["""That movie was absolutely awful."""])
print('Predictions: {}'.format(predictions))
print('Raw outputs: {}'.format(raw_outputs))